In [41]:
import numpy as np
import pandas as pd

In [42]:
data = pd.read_csv('/content/mobile_price_classification (1).csv')
data.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [43]:
data.shape

(2000, 21)

In [44]:
data['price_range'].value_counts()

1    500
2    500
3    500
0    500
Name: price_range, dtype: int64

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   battery_power   2000 non-null   int64  
 1   bluetooth       2000 non-null   int64  
 2   clock_speed     2000 non-null   float64
 3   dual_sim        2000 non-null   int64  
 4   front_cam       2000 non-null   int64  
 5   4G              2000 non-null   int64  
 6   int_memory      2000 non-null   int64  
 7   m_dep           2000 non-null   float64
 8   mobile_wt       2000 non-null   int64  
 9   n_cores         2000 non-null   int64  
 10  primary_camera  2000 non-null   int64  
 11  px_height       2000 non-null   int64  
 12  px_width        2000 non-null   int64  
 13  ram             2000 non-null   int64  
 14  sc_h            2000 non-null   int64  
 15  sc_w            2000 non-null   int64  
 16  talk_time       2000 non-null   int64  
 17  three_g         2000 non-null   i

In [46]:
data.isnull().sum()

battery_power     0
bluetooth         0
clock_speed       0
dual_sim          0
front_cam         0
4G                0
int_memory        0
m_dep             0
mobile_wt         0
n_cores           0
primary_camera    0
px_height         0
px_width          0
ram               0
sc_h              0
sc_w              0
talk_time         0
three_g           0
touch_screen      0
wifi              0
price_range       0
dtype: int64

In [47]:
x = data.drop('price_range',axis=1)
y = data['price_range']

In [48]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_trian,y_test = train_test_split(x,y,test_size =0.2,random_state=42)

In [49]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [50]:

def create_model(optimizer = 'SGD'):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
  model.add(Dense(4,activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])
  return model

In [51]:
seed = 7
tf.random.set_seed(seed)

In [52]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [53]:
model = KerasClassifier(model = create_model, batch_size = 21, verbose = 0)

In [54]:
# define the grid search parameters
optimizer = ['SGD',  'Adam', 'Adamax', 'Nadam']
epochs = [10,20,30]
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(model__optimizer=optimizer,optimizer__learning_rate=learn_rate,epochs=epochs)

In [55]:
param_grid

{'model__optimizer': ['SGD', 'Adam', 'Adamax', 'Nadam'],
 'optimizer__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
 'epochs': [10, 20, 30]}

In [56]:
grid = GridSearchCV(estimator=model,param_grid = param_grid,n_jobs =-1,cv=3)
grid_result = grid.fit(x,y)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [58]:
best_params = grid_result.best_params_
best_model = grid_result.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Best Parameters:", best_params)
print("Test Accuracy:", accuracy)

Best Parameters: {'epochs': 30, 'model__optimizer': 'Nadam', 'optimizer__learning_rate': 0.3}
Test Accuracy: 0.645
